# **The Storm Of the Century: Geopotential Height ERA5**
***

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
import metpy.calc as mpcalc
from scipy import ndimage  
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
import metpy
from metpy.plots import ctables
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
reanalysis = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2', 
    consolidated=True
)

In [ ]:
reanalysis

## **Date Input**

In [ ]:
startYear = 2011
startMonth = 10
startDay = 7
startHour = 0
startMinute = 0
startDateTime = dt(startYear,startMonth,startDay, startHour, startMinute)

endYear = 1993
endMonth = 3
endDay = 15
endHour = 0
endMinute = 0
endDateTime = dt(endYear,endMonth,endDay, endHour, endMinute)

In [ ]:
dateList = pd.date_range(startDateTime, endDateTime,freq="6H")
dateList

## **ERA-5**

## **Area and Domain**

In [ ]:
lonW = -99 + 360
lonE = -59 + 360
latS = 20
latN = 50
inc=.25
cLat, cLon = (latS + latN)/2, (lonW + lonE)/2
latRange = np.arange(latN,latS-inc,-inc) # expand the data range a bit beyond the plot range
lonRange = np.arange(lonW,lonE+inc,+inc) # Need to match longitude values to those of the coordinate variable
constrainLat, constrainLon = (0.6, 7)
proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree() # Our data is lat-lon; thus its native projection is Plate Carree.
res = '50m'

## **Variables**

In [ ]:
plevel=500
Z = reanalysis['geopotential'].sel(time=dateList,level=plevel,latitude=latRange,longitude=lonRange)

In [ ]:
Z

In [ ]:
lats = Z.latitude
lons = Z.longitude

## **MetPy Calculations**

In [ ]:
HGHT = mpcalc.geopotential_to_height(Z)

In [ ]:
HGHT

In [ ]:
HGHT = HGHT.metpy.convert_units('dam')

In [ ]:
HGHT = HGHT.compute()

In [ ]:
HGHT.min(),HGHT.max()

## **Graphing**

In [ ]:
contour_int = 6
HGHTContours = np.arange (462, 606, contour_int)

In [ ]:
def segment_cmap(cmap_name, start, end):
    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(start, end))
    return LinearSegmentedColormap.from_list(f"{cmap_name}_segment_{start}_{end}", colors)



In [ ]:
#segment_cmap = segment_cmap('RdPu', 0.4, 1)

In [ ]:
cmap='jet'
bounds = np.arange(490, 600, 10)
ticks = bounds


In [ ]:
#cmap_colors = cmap(np.linspace(0, 1, cmap.N))
#n_transparent = 50  # Number of first entries to make transparent
#cmap_colors[:n_transparent, -1] = np.linspace(0, 1, n_transparent)  # Adjust alpha from 0 to 1

# Create a new colormap with the modified alpha
#transparent_cmap = LinearSegmentedColormap.from_list('PuRd_transparent', cmap_colors)
#cmap=transparent_cmap

In [ ]:
for time in dateList:
    print("Processing", time)
    
    # Create the time strings, for the figure title as well as for the file name.
    timeStr = dt.strftime(time,format="%Y-%m-%d %H%M UTC")
    timeStrFile = dt.strftime(time,format="%Y%m%d%H")
    levstr=f'{plevel}'
    
    tl1 = str('ERA5, Valid at: '+ timeStr)
    tl2 = ("Geopotential Height (dam) at " + levstr + "mb")
    
    title_line = (tl1 + '\n' + tl2 + '\n')
    
    fig = plt.figure(figsize=(22,14)) # Increase size to adjust for the constrained lats/lons
    ax = plt.subplot(1,1,1,projection=proj_map)
    ax.set_extent ([lonW+constrainLon,lonE-constrainLon,latS+constrainLat,latN-constrainLat])
    ax.add_feature(cfeature.COASTLINE.with_scale(res))
    ax.add_feature(cfeature.STATES.with_scale(res),edgecolor='brown')
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature (cfeature.RIVERS.with_scale(res))
    ax.add_feature (cfeature.LAND.with_scale(res))
    ax.add_feature (cfeature.COASTLINE.with_scale(res))
    ax.add_feature (cfeature.LAKES.with_scale(res))
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature(cfeature.OCEAN)
    
    cZ= ax.contour(lons, lats, HGHT.sel(time=time), levels=HGHTContours, colors='black', linewidths=3, transform=proj_data)
    ax.clabel(cZ, inline=1, fontsize=12, fmt='%.0f')
    
    contourf=ax.contourf(lons,lats,HGHT.sel(time=time),levels=HGHTContours,cmap=cmap,transform=proj_data,extend='both')
    cbar= plt.colorbar(contourf, ax=ax, ticks=ticks)
    cbar.set_label('Geopotential Height dam', rotation=270, labelpad=15)
    
    
    # Wind barbs
    #skip = 4
    #ax.barbs(lons[::skip],lats[::skip],UKts.sel(time=time)[::skip,::skip].values, VKts.sel(time=time)[::skip,::skip].values, color='black',zorder=2,transform=proj_data)

    title = plt.title(title_line,fontsize=16)